In [149]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

OK !


In [150]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor
from haystack.document_stores import ElasticsearchDocumentStore

print('OK !')

OK !


In [151]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

preprocessor_100 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

preprocessor_200 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
)

preprocessor_300 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=300,
    split_respect_sentence_boundary=True,
)

print('OK !')

OK !


# Set preprocessor 

In [152]:
preprocessor = preprocessor_100

INFO - backoff -  Backing off send_request(...) for 0.0s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - backoff -  Backing off send_request(...) for 0.8s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))


# Process Raw PDF

In [153]:
# root = '../shared_data/thanatcc/RIA_Input/International Regulator'
root = '/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator'


file_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
#         print(os.path.join(path, name))
        file_list.append(os.path.join(path, name))
    
len(file_list)
# file_list

file_list = [each for each in file_list if '.pdf' in each or '.PDF' in each]

len(file_list)

INFO - backoff -  Backing off send_request(...) for 3.8s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))


384

384

# Clean File Name

In [154]:
def clean_file_name(content):
    content = content.replace('\n',' ')
    content = content.encode('utf-8', 'replace').decode()
    content = content.encode('utf8','surrogateescape').decode('utf8','surrogateescape')
    content = content.replace('?','')
    content = content.replace(',','')
    content = " ".join(content.split())
    content = content.replace(' ','_')
    content = content.replace('__','_')
    return content

import unicodedata
import re

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

print('OK !')

OK !


In [155]:
clean_file_list = []
for file_name in file_list:
    clean_file_name = file_name.split('/')[-1]
    clean_file_name = clean_file_name.replace('.pdf', '')
    clean_file_name = clean_file_name.replace('.PDF', '')
    clean_file_name = slugify(clean_file_name)
    clean_file_name = clean_file_name + '.pdf'
    dir_name = '/'.join(file_name.split('/')[:-1])
    clean_file_list.append(dir_name + '/' + clean_file_name)
    
clean_file_list = ['/'.join((each.split('/')[-3:])) for each in clean_file_list]

ERROR - backoff -  Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
ERROR - posthog -  error uploading: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))


In [156]:
des_path = '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/'

clean_dir_list = ['/'.join((each.split('/')[:2])) for each in clean_file_list]
clean_dir_list = list(set(clean_dir_list))
clean_dir_list = [each for each in clean_dir_list]
clean_dir_list = [des_path + each for each in clean_dir_list]
for each in clean_dir_list:
    os.makedirs(each, exist_ok=True)

In [157]:
clean_file_list = [des_path + each for each in clean_file_list]

In [161]:
import shutil

clean_file_list = [each.replace('/International Regulator/', '/Clean_International_Regulator/') for each in clean_file_list]

for i, clean_file_name in enumerate(clean_file_list):
    print(i)
    shutil.copyfile(file_list[i], clean_file_name)
    print(file_list[i],'\n', clean_file_name)

0


'../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/10-FCA/interest rate/ps14-16.pdf'

/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator/10-FCA/interest rate/ps14-16.pdf 
 ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/10-FCA/interest rate/ps14-16.pdf
1


'../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/10-FCA/interest rate/ps18-19.pdf'

/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator/10-FCA/interest rate/ps18-19.pdf 
 ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/10-FCA/interest rate/ps18-19.pdf
2


...

/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator/26-BNM/interest rate/Revised Reference Rate Framework - Bank Negara Malaysia.pdf 
 ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/26-BNM/interest rate/revised-reference-rate-framework-bank-negara-malaysia.pdf
382


'../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/26-BNM/interest rate/pd-credit-card.pdf'

/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator/26-BNM/interest rate/PD Credit Card.pdf 
 ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/26-BNM/interest rate/pd-credit-card.pdf
383


'../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/99-Other/interest rate/loans-and-credit-cards-_-asic-australian-securities-and-investments-commission.pdf'

/DA_WORKSPACE/GLOBAL_WS/ria/International Regulator/99-Other/interest rate/Loans and credit cards _ ASIC - Australian Securities and Investments Commission.pdf 
 ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/99-Other/interest rate/loans-and-credit-cards-_-asic-australian-securities-and-investments-commission.pdf


In [162]:
converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages = ["en"])

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC
INFO - backoff -  Backing off send_request(...) for 0.1s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))


# Detect ENG Language

In [163]:
from langdetect import detect

print('OK !')

OK !


INFO - backoff -  Backing off send_request(...) for 1.1s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - backoff -  Backing off send_request(...) for 2.4s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))


In [164]:
root = '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator'

clean_file_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
#         print(os.path.join(path, name))
        clean_file_list.append(os.path.join(path, name))
    
len(clean_file_list)
# file_list

clean_file_list = [each for each in clean_file_list if '.pdf' in each or '.PDF' in each]

len(clean_file_list)

384

384

ERROR - backoff -  Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
ERROR - posthog -  error uploading: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))
INFO - backoff -  Backing off send_request(...) for 0.7s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))

In [165]:
doc_list = []
error_list = []

for file_name in tqdm(clean_file_list):
    try:
        doc = converter.convert(file_path = file_name, meta = None)
        docs = preprocessor.process(doc[0])
        lang_list = []
        for i in range(len(docs)): 
            try:
                temp_detect = detect(docs[i]['content'])
                lang_list.append(temp_detect)
            except:
                pass
        en_df = pd.DataFrame(lang_list)[0].value_counts(normalize = True).reset_index()
        en_score = float(en_df[en_df['index'] == 'en'][0].values[0])

        if float(en_score) < 0.9:
            error_list.append(file_name)
            print('Error:', file_name)
    except:
#         break
        error_list.append(file_name)
        print('Error:', file_name)
        pass

  0%|                                                                                                                                                                                   | 0/384 [00:00<?, ?it/s]INFO - backoff -  Backing off send_request(...) for 3.9s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
WARNING - haystack.nodes.preprocessor.preprocessor -  One or more sentence found with word count higher than the split length.
  1%|█▎                                                                                                                                                                         | 3/384 [00:04<07:36,  1.20s/it]Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight

Error: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/03-BCBS/risk/ripple-effects-of-monetary-policy.pdf


 23%|███████████████████████████████████████▍                                                                                                                                  | 89/384 [01:02<03:43,  1.32it/s]

Error: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_346.pdf


 27%|████████████████████████████████████████████▉                                                                                                                            | 102/384 [01:07<01:56,  2.41it/s]Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <21> in hex string
Syntax Error (4): Illegal character <4f> in hex string
Syntax Error (6): Illegal character <54> in hex string
Syntax Error (7): Illegal character <59> in hex string
Syntax Error (8): Illegal character <50> in hex string
Syntax Error (11): Illegal character <68> in hex string
Syntax Error (12): Illegal character <74> in hex string
Syntax Error (13): Illegal character <6d> in hex string
Syntax Error (14): Illegal character <6c> in hex string
Syntax Error (19): Illegal character <68> in hex string
Syntax Error (20): Illegal character <74> in hex string
Syntax Error (21): Illegal character <6d> in hex string
Syntax Error (22): Illegal character <6c> in hex string
Syntax Erro

...

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 352/384 [02:34<00:07,  4.20it/s]Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <21> in hex string
Syntax Error (4): Illegal character <4f> in hex string
Syntax Error (6): Illegal character <54> in hex string
Syntax Error (7): Illegal character <59> in hex string
Syntax Error (8): Illegal character <50> in hex string
Syntax Error (11): Illegal character <68> in hex string
Syntax Error (12): Illegal character <74> in hex string
Syntax Error (13): Illegal character <6d> in hex string
Syntax Error (14): Illegal character <6c> in hex string
Syntax Error (19): Illegal character <68> in hex string
Syntax Error (20): Illegal character <74> in hex string
Syntax Error (21): Illegal character <6d> in hex string
Syntax Error (22): Illegal character <6c> in hex string
Syntax Erro

Error: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_294.pdf


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍   | 376/384 [02:40<00:02,  3.28it/s]

Error: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/digital assets/financial-services-and-markets-bill-uthority-of-singapore-on-4-april-2022.pdf


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384/384 [03:01<00:00,  2.12it/s]


In [170]:
error_list

['../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/03-BCBS/risk/ripple-effects-of-monetary-policy.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_346.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_191.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_246.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_235.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_192.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_290.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_193.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_56.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_289.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_Internationa

In [171]:
len(clean_file_list)

clean_file_list = [each for each in clean_file_list if each not in error_list]

len(clean_file_list)

372

372

# OCR Error List

In [183]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from PyPDF2 import PdfFileMerger
from progressbar import ProgressBar
import os

print('OK !')

OK !


In [184]:
pytesseract.get_tesseract_version()

LooseVersion ('4.1.1')

In [186]:
dir_list = error_list
dir_list = ['/'.join((each.split('/')[:-1])) for each in dir_list]
dir_list = list(set(dir_list))
dir_list = [each.replace('/Clean_International_Regulator/', '/Clean_International_Regulator_OCR/') for each in dir_list]

In [188]:
for each in dir_list:
    os.makedirs(each, exist_ok=True)

In [189]:
root = '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator_OCR'

ocr_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
        ocr_list.append(os.path.join(path, name))
    
len(ocr_list)

ocr_list = [each for each in ocr_list if '.pdf' in each or '.PDF' in each]

len(ocr_list)

0

0

In [190]:
error_list_2 = []
for file_name in tqdm(error_list):
    try:
        output_path = file_name.replace('/Clean_International_Regulator/', '/Clean_International_Regulator_OCR/')
        if output_path not in ocr_list:
            print('Start:', output_path)
            doc = convert_from_path(file_name)
            #doc = doc[:5]
            merger = PdfFileMerger()
            pbar=ProgressBar()
            for page_index, image in enumerate(pbar(doc)):
                pdf_page = pytesseract.image_to_pdf_or_hocr(image, extension='pdf')
                pdf_page_path = f"/tmp/{page_index}.pdf"        
                with open(pdf_page_path, "wb") as f:
                    f.write(pdf_page)
                merger.append(pdf_page_path)
                os.remove(pdf_page_path)
            merger.write(output_path)
            print('Success:', output_path)
    except:
        print('Error:', file_name)
        error_list_2.append(file_name)
        pass

  0%|                                                                                                                                                                                    | 0/12 [00:00<?, ?it/s]

Start: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator_OCR/03-BCBS/risk/ripple-effects-of-monetary-policy.pdf


154041

164202

423609

...

780302

813251

108280

100% |#########################################################################################################################################################################################################|
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [05:01<00:00, 25.13s/it]

Success: ../shared_data/thanatcc/RIA_Input/Clean_International_Regulator_OCR/25-MAS/digital assets/financial-services-and-markets-bill-uthority-of-singapore-on-4-april-2022.pdf


In [191]:
error_list_2

['../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_191.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_192.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_193.pdf',
 '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator/25-MAS/other/sg_294.pdf']

# Export valid_file_list

In [192]:
root = '../shared_data/thanatcc/RIA_Input/Clean_International_Regulator_OCR'

ocr_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
        ocr_list.append(os.path.join(path, name))
    
len(ocr_list)

ocr_list = [each for each in ocr_list if '.pdf' in each or '.PDF' in each]

len(ocr_list)

8

8

In [193]:
valid_file_list = clean_file_list

len(valid_file_list)

valid_file_list = [each for each in valid_file_list if each not in error_list]

len(valid_file_list)

valid_file_list = valid_file_list + ocr_list

len(valid_file_list)

372

372

380

In [194]:
import pickle
with open('data/valid_file_list.pickle', 'wb') as handle:
    pickle.dump(valid_file_list, handle)

In [124]:
# valid_file_list